In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../data/processed/spx500_2017-2024_notnull.csv").drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
df.head()

,date,symbol,open,high,low,close,volume,adj_close
0,2017-01-03,A,45.930000,46.750000,45.740002,46.490002,1739600.0,43.975956
1,2017-01-03,AAL,47.279999,47.340000,46.139999,46.299999,6737800.0,44.741226
2,2017-01-03,AAPL,28.950001,29.082500,28.690001,29.037500,115127600.0,26.952707
3,2017-01-03,ABBV,62.919998,63.029999,61.939999,62.410000,9328200.0,45.028652
4,2017-01-03,ABT,38.630001,39.080002,38.340000,39.049999,9677300.0,34.127411


In [14]:
# the source dataframe has been sorted by date; otherwise using df.sorted_values(by=['date'])
((df[df.symbol=="AAPL"].date <= df[df.symbol=="AAPL"].date.shift(1))).sum()

0

In [5]:
# generate factors according to GPT-4o
df['return'] = df.groupby('symbol')['adj_close'].pct_change()
df['MA_20'] = df.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).mean())
df['MA_50'] = df.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=50).mean())

df['EMA_12'] = df.groupby('symbol')['adj_close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
df['EMA_26'] = df.groupby('symbol')['adj_close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['Signal_Line'] = df.groupby('symbol')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
# generate MACD cross signal
df['MACD_Buy_Signal'] = ((df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1))).astype(int)
df['MACD_Sell_Signal'] = ((df['MACD'] < df['Signal_Line']) & (df['MACD'].shift(1) >= df['Signal_Line'].shift(1))).astype(int)

# generate Bollinger Band breakthrough signal
df['BB_Middle'] = df.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).mean())
df['BB_Upper'] = df['BB_Middle'] + 2 * df.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).std())
df['BB_Lower'] = df['BB_Middle'] - 2 * df.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).std())
df['BB_Buy_Signal'] = df['adj_close'] - df['BB_Lower']  # <
df['BB_Sell_Signal'] = df['adj_close'] - df['BB_Upper']  # >

# calculate the relative strength index (RSI)
def RSI(series, period=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI_14'] = df.groupby('symbol')['adj_close'].transform(lambda x: RSI(x, 14))

In [4]:
# df.to_csv("../data/processed/spx500_MACD_factors.csv")

In [27]:
df['return'].describe()

count    896368.000000
mean          0.000682
std           0.021253
min          -0.538647
25%          -0.008439
50%           0.000864
75%           0.009954
max           0.745932
Name: return, dtype: float64

In [22]:
df.tail()

,date,symbol,open,high,low,close,volume,adj_close
896847,2024-05-14,XYL,141.259995,142.720001,140.929993,142.070007,949800.0,141.699249
896848,2024-05-14,YUM,136.520004,138.449997,136.190002,137.490005,2180300.0,136.817657
896849,2024-05-14,ZBH,119.949997,120.510002,118.330002,119.970001,1348700.0,119.970001
896850,2024-05-14,ZBRA,318.390015,322.779999,317.850006,320.980011,216300.0,320.980011
896851,2024-05-14,ZTS,169.809998,170.199997,168.630005,168.869995,2050100.0,168.869995


In [26]:
df.shape

(896852, 23)

In [2]:
# machine learning process
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction.settings import EfficientFCParameters

In [6]:
df_2017 = df[(df['date'] >= '2017-01-01') & (df['date'] <= '2017-12-30')].copy()
df_2017 = df_2017.dropna(axis='index')
df_2017

,date,symbol,open,high,low,close,volume,adj_close,return,MA_20,...,MACD,Signal_Line,MACD_Buy_Signal,MACD_Sell_Signal,BB_Middle,BB_Upper,BB_Lower,BB_Buy_Signal,BB_Sell_Signal,RSI_14
23716,2017-03-15,A,53.310001,54.820000,53.020000,54.099998,4118000.0,51.174427,0.036200,48.930700,...,0.761991,0.687741,0,0,48.930700,50.224377,47.637023,3.537404,0.950050,69.196990
23717,2017-03-15,AAL,40.500000,42.189999,40.209999,42.090000,13986200.0,40.763428,0.021354,43.929400,...,-1.009710,-0.540259,0,1,43.929400,47.510446,40.348354,0.415074,-6.747018,32.082159
23718,2017-03-15,AAPL,34.852501,35.187500,34.757500,35.115002,102767200.0,32.735188,0.010576,32.166056,...,0.678252,0.785404,0,0,32.166056,32.919558,31.412554,1.322634,-0.184370,72.929014
23719,2017-03-15,ABBV,65.699997,66.790001,65.680000,66.550003,5241200.0,48.500126,0.013400,46.110825,...,0.872156,0.600736,1,0,46.110825,48.541038,43.680611,4.819515,-0.040913,89.810125
23720,2017-03-15,ABT,44.959999,45.400002,44.860001,45.320000,6844400.0,39.862858,0.011607,39.737514,...,0.630376,0.784274,0,1,39.737514,40.576189,38.898840,0.964018,-0.713331,49.078402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121479,2017-12-29,XYL,68.529999,68.800003,67.919998,68.199997,1046700.0,63.057022,-0.000733,62.772249,...,0.259982,0.290593,0,1,62.772249,63.735682,61.808815,1.248207,-0.678660,50.113298
121480,2017-12-29,YUM,82.639999,82.709999,81.589996,81.610001,1347600.0,72.585533,-0.012822,73.477627,...,0.500547,0.731247,0,0,73.477627,74.415278,72.539976,0.045557,-1.829745,35.630091
121481,2017-12-29,ZBH,118.203880,118.398056,117.106796,117.155342,1054308.0,111.635948,-0.008870,107.909759,...,1.626387,1.084024,1,0,107.909759,114.042029,101.777488,9.858460,-2.406081,73.631006
121482,2017-12-29,ZBRA,104.330002,104.940002,103.279999,103.800003,159500.0,103.800003,-0.004603,105.884000,...,-1.061514,-1.011452,0,1,105.884000,110.116732,101.651268,2.148735,-6.316729,35.140981


In [33]:
df_2017.isnull().sum()

date                0
symbol              0
open                0
high                0
low                 0
close               0
volume              0
adj_close           0
return              0
MA_20               0
MA_50               0
EMA_12              0
EMA_26              0
MACD                0
Signal_Line         0
MACD_Buy_Signal     0
MACD_Sell_Signal    0
BB_Middle           0
BB_Upper            0
BB_Lower            0
BB_Buy_Signal       0
BB_Sell_Signal      0
RSI_14              0
dtype: int64

In [7]:
# clustering according to 2017 time series: using tsfreshing package
features_df = df_2017[['date', 'symbol', 'adj_close', 'volume', 'return']]
# Extracting features
extracted_features = extract_features(
    features_df,
    column_id='symbol',
    column_sort='date',
    default_fc_parameters=EfficientFCParameters(),
    n_jobs=4
)

# Handling NaN values in extracted features
extracted_features.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace inf values with NaN
extracted_features.dropna(axis=1, inplace=True)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(extracted_features)

kmeans = KMeans(n_clusters=5, random_state=42)  # supposed divided into 5 groups
extracted_features['cluster'] = kmeans.fit_predict(scaled_features)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:32<00:00,  1.65s/it]


In [9]:
extracted_features

,return__variance_larger_than_standard_deviation,return__has_duplicate_max,return__has_duplicate_min,return__has_duplicate,return__sum_values,return__abs_energy,return__mean_abs_change,return__mean_change,return__mean_second_derivative_central,return__median,...,volume__fourier_entropy__bins_5,volume__fourier_entropy__bins_10,volume__fourier_entropy__bins_100,volume__permutation_entropy__dimension_3__tau_1,volume__permutation_entropy__dimension_4__tau_1,volume__permutation_entropy__dimension_5__tau_1,volume__permutation_entropy__dimension_6__tau_1,volume__permutation_entropy__dimension_7__tau_1,volume__mean_n_absolute_max__number_of_maxima_7,cluster
A,0.0,0.0,0.0,1.0,0.268308,0.020867,0.011157,-0.000205,0.000121,0.001156,...,0.775286,1.395155,3.293749,1.772580,3.061756,4.303607,5.026283,5.256896,5.188429e+06,1
AAL,0.0,0.0,0.0,1.0,0.270539,0.062312,0.018982,-0.000147,0.000028,0.000646,...,0.768332,1.406273,3.421808,1.761741,3.058885,4.307991,4.973437,5.197643,1.395406e+07,1
AAPL,0.0,0.0,0.0,0.0,0.221411,0.025914,0.011880,-0.000106,-0.000012,0.000342,...,0.274751,0.756124,2.788958,1.746685,3.043404,4.330459,5.071162,5.256896,2.525477e+08,4
ABBV,0.0,0.0,0.0,1.0,0.426395,0.027381,0.010779,-0.000122,0.000038,0.001622,...,0.410762,0.667848,2.464531,1.786596,3.152431,4.492207,5.118696,5.249823,1.465550e+07,1
ABT,0.0,0.0,0.0,1.0,0.266563,0.016056,0.010050,-0.000092,0.000031,0.000290,...,0.528971,0.987858,2.926352,1.782705,3.110660,4.422192,5.092273,5.249823,1.299360e+07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.0,0.0,0.0,0.0,0.369089,0.019648,0.010649,-0.000056,0.000014,0.001180,...,0.467999,0.976129,2.870651,1.782969,3.096023,4.375688,5.040358,5.242750,2.483057e+06,1
YUM,0.0,0.0,0.0,0.0,0.264335,0.016118,0.008647,-0.000061,-0.000038,0.000717,...,0.509883,1.146288,3.151978,1.739124,3.049343,4.345130,5.009553,5.214458,5.348314e+06,1
ZBH,0.0,0.0,0.0,0.0,0.039967,0.030688,0.012136,-0.000085,-0.000035,0.000885,...,0.428809,0.951551,2.947558,1.772048,3.084723,4.346892,5.083511,5.242750,4.498407e+06,0
ZBRA,0.0,0.0,0.0,1.0,0.180598,0.052528,0.016535,-0.000133,0.000048,0.000924,...,0.399363,0.919199,2.848422,1.778066,3.100978,4.379956,5.109003,5.242750,9.235143e+05,1


In [10]:
# extracted_features.to_csv("../data/processed/cluster_tsfresh.csv")

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [10]:
extracted_features = pd.read_csv("../data/processed/cluster_tsfresh.csv").rename(columns={'Unnamed: 0': 'symbol'}).set_index('symbol')
# extracted_features.reset_index(inplace=True)
extracted_features

,return__variance_larger_than_standard_deviation,return__has_duplicate_max,return__has_duplicate_min,return__has_duplicate,return__sum_values,return__abs_energy,return__mean_abs_change,return__mean_change,return__mean_second_derivative_central,return__median,...,volume__fourier_entropy__bins_5,volume__fourier_entropy__bins_10,volume__fourier_entropy__bins_100,volume__permutation_entropy__dimension_3__tau_1,volume__permutation_entropy__dimension_4__tau_1,volume__permutation_entropy__dimension_5__tau_1,volume__permutation_entropy__dimension_6__tau_1,volume__permutation_entropy__dimension_7__tau_1,volume__mean_n_absolute_max__number_of_maxima_7,cluster
symbol,,,,,,,,,,,,,,,,,,,,,
A,0.0,0.0,0.0,1.0,0.268308,0.020867,0.011157,-0.000205,0.000121,0.001156,...,0.775286,1.395155,3.293749,1.772580,3.061756,4.303607,5.026283,5.256896,5.188429e+06,1
AAL,0.0,0.0,0.0,1.0,0.270539,0.062312,0.018982,-0.000147,0.000028,0.000646,...,0.768332,1.406273,3.421808,1.761741,3.058885,4.307991,4.973437,5.197643,1.395406e+07,1
AAPL,0.0,0.0,0.0,0.0,0.221411,0.025914,0.011880,-0.000106,-0.000012,0.000342,...,0.274751,0.756124,2.788958,1.746685,3.043404,4.330459,5.071162,5.256896,2.525477e+08,4
ABBV,0.0,0.0,0.0,1.0,0.426395,0.027381,0.010779,-0.000122,0.000038,0.001622,...,0.410762,0.667848,2.464531,1.786596,3.152431,4.492207,5.118696,5.249823,1.465550e+07,1
ABT,0.0,0.0,0.0,1.0,0.266563,0.016056,0.010050,-0.000092,0.000031,0.000290,...,0.528971,0.987858,2.926352,1.782705,3.110660,4.422192,5.092273,5.249823,1.299360e+07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.0,0.0,0.0,0.0,0.369089,0.019648,0.010649,-0.000056,0.000014,0.001180,...,0.467999,0.976129,2.870651,1.782969,3.096023,4.375688,5.040358,5.242750,2.483057e+06,1
YUM,0.0,0.0,0.0,0.0,0.264335,0.016118,0.008647,-0.000061,-0.000038,0.000717,...,0.509883,1.146288,3.151978,1.739124,3.049343,4.345130,5.009553,5.214458,5.348314e+06,1
ZBH,0.0,0.0,0.0,0.0,0.039967,0.030688,0.012136,-0.000085,-0.000035,0.000885,...,0.428809,0.951551,2.947558,1.772048,3.084723,4.346892,5.083511,5.242750,4.498407e+06,0


In [11]:
# Handling NaN values in extracted features
extracted_features.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace inf values with NaN
extracted_features.dropna(axis=1, inplace=True)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(extracted_features)

kmeans = KMeans(n_clusters=5, random_state=42)  # supposed divided into 5 groups
extracted_features['cluster'] = kmeans.fit_predict(scaled_features)

extracted_features.reset_index(inplace=True)

In [12]:
# Outputting the symbol lists for each cluster
clusters = extracted_features.groupby('cluster')['symbol'].apply(list)
for i, cluster in enumerate(clusters):
    print(f"Cluster {i} symbols: {cluster}")

Cluster 0 symbols: ['ACGL', 'ADM', 'ADSK', 'AES', 'AIG', 'AKAM', 'APA', 'AXON', 'AZO', 'BBWI', 'BBY', 'BG', 'BIO', 'BKNG', 'BKR', 'BSX', 'BXP', 'CAG', 'CAH', 'CHD', 'CHTR', 'CMCSA', 'CMG', 'COR', 'CPB', 'CSX', 'CVS', 'DECK', 'DGX', 'DIS', 'DOC', 'DPZ', 'DRI', 'DVA', 'DVN', 'DXCM', 'EFX', 'EG', 'EIX', 'ENPH', 'EQT', 'EXPE', 'FANG', 'FAST', 'FCX', 'FFIV', 'FRT', 'GE', 'GEN', 'GIS', 'GPC', 'GWW', 'HAL', 'HAS', 'HCA', 'HES', 'HOLX', 'HPE', 'HRL', 'HSIC', 'HWM', 'IBM', 'IDXX', 'INCY', 'IPG', 'JBL', 'JCI', 'JNPR', 'K', 'KDP', 'KHC', 'KIM', 'KLAC', 'KMB', 'KMI', 'KR', 'LOW', 'LULU', 'MCK', 'MDLZ', 'MDT', 'MKC', 'MKTX', 'MLM', 'MO', 'MOH', 'MOS', 'MRK', 'MRO', 'MU', 'NCLH', 'NDSN', 'NKE', 'NRG', 'NUE', 'O', 'OKE', 'OMC', 'ORCL', 'ORLY', 'PARA', 'PCG', 'PM', 'POOL', 'PPL', 'PSA', 'QCOM', 'QRVO', 'REGN', 'SBUX', 'SJM', 'SLB', 'SMCI', 'SNA', 'SPG', 'STX', 'SWKS', 'SYF', 'T', 'TAP', 'TGT', 'TJX', 'TMUS', 'TPR', 'TRGP', 'TSCO', 'UAL', 'UHS', 'ULTA', 'VMC', 'VRTX', 'VST', 'VTRS', 'WAB', 'WBA', 'WBD'

In [ ]:
extracted_features['pca-one'] = pca_result[:, 0]
extracted_features['pca-two'] = pca_result[:, 1]

# Visualization with Seaborn
plt.figure(figsize=(16, 10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="cluster",
    palette=sns.color_palette("hsv", 5),
    data=extracted_features,
    legend="full",
    alpha=0.8
)
plt.title('PCA of Clustered Features')
plt.show()

In [ ]:
# Using DBSCAN to ensure minimum number of points in each cluster
dbscan = DBSCAN(eps=0.5, min_samples=5)
extracted_features['dbscan_cluster'] = dbscan.fit_predict(scaled_features)

In [ ]:
# clustering according to 2017 time series: using correlation